# A00822830 Donaldo Salazar

In [ ]:
!pip install praw

In [1]:
import nltk
import random
from nltk.classify.scikitlearn import SklearnClassifier
import pickle
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from nltk.classify import ClassifierI
from statistics import mode
from nltk.tokenize import word_tokenize


class VoteClassifier(ClassifierI):
    def __init__(self, *classifiers):
        self._classifiers = classifiers

    def classify(self, features):
        votes = []
        for c in self._classifiers:
            v = c.classify(features)
            votes.append(v)
        return mode(votes)

    def confidence(self, features):
        votes = []
        for c in self._classifiers:
            v = c.classify(features)
            votes.append(v)

        choice_votes = votes.count(mode(votes))
        conf = choice_votes / len(votes)
        return conf


short_pos = open("short_reviews/positive.txt", "r").read()
short_neg = open("short_reviews/negative.txt", "r").read()


all_words = []
documents = []


allowed_word_types = ["J"]

for p in short_pos.split('\n'):
    documents.append((p, "pos"))
    words = word_tokenize(p)
    pos = nltk.pos_tag(words)
    for w in pos:
        if w[1][0] in allowed_word_types:
            all_words.append(w[0].lower())

for p in short_neg.split('\n'):
    documents.append((p, "neg"))
    words = word_tokenize(p)
    pos = nltk.pos_tag(words)
    for w in pos:
        if w[1][0] in allowed_word_types:
            all_words.append(w[0].lower())
            
all_words = nltk.FreqDist(all_words)

word_features = list(all_words.keys())[:5000]

def find_features(document):
    words = word_tokenize(document)
    features = {}
    for w in word_features:
        features[w] = (w in words)

    return features


featuresets = [(find_features(rev), category) for (rev, category) in documents]

random.shuffle(featuresets)
print(len(featuresets))

testing_set = featuresets[10000:]
training_set = featuresets[:10000]

10664


In [2]:
BernoulliNB_classifier = SklearnClassifier(BernoulliNB())
BernoulliNB_classifier.train(training_set)
print("BernoulliNB_classifier accuracy percent:", (nltk.classify.accuracy(BernoulliNB_classifier, testing_set)) * 100)

save_classifier = open("pickled_algos/BernoulliNB_classifier5k.pickle", "wb")
pickle.dump(BernoulliNB_classifier, save_classifier)
save_classifier.close()

BernoulliNB_classifier accuracy percent: 73.94578313253012


In [3]:
LogisticRegression_classifier = SklearnClassifier(LogisticRegression())
LogisticRegression_classifier.train(training_set)
print("LogisticRegression_classifier accuracy percent:", (nltk.classify.accuracy(LogisticRegression_classifier, testing_set)) * 100)

save_classifier = open("pickled_algos/LogisticRegression_classifier5k.pickle", "wb")
pickle.dump(LogisticRegression_classifier, save_classifier)
save_classifier.close()

LogisticRegression_classifier accuracy percent: 69.87951807228916


In [4]:
LinearSVC_classifier = SklearnClassifier(LinearSVC())
LinearSVC_classifier.train(training_set)
print("LinearSVC_classifier accuracy percent:", (nltk.classify.accuracy(LinearSVC_classifier, testing_set)) * 100)

save_classifier = open("pickled_algos/LinearSVC_classifier5k.pickle", "wb")
pickle.dump(LinearSVC_classifier, save_classifier)
save_classifier.close()

LinearSVC_classifier accuracy percent: 69.27710843373494


In [5]:
SGDC_classifier = SklearnClassifier(SGDClassifier())
SGDC_classifier.train(training_set)
print("SGDClassifier accuracy percent:", nltk.classify.accuracy(SGDC_classifier, testing_set) * 100)

save_classifier = open("pickled_algos/SGDC_classifier5k.pickle", "wb")
pickle.dump(SGDC_classifier, save_classifier)
save_classifier.close()

SGDClassifier accuracy percent: 68.07228915662651


In [6]:
classifier = nltk.NaiveBayesClassifier.train(training_set)
print("Original Naive Bayes Algo accuracy percent:", (nltk.classify.accuracy(classifier, testing_set))*100)
classifier.show_most_informative_features(15)

###############
save_classifier = open("pickled_algos/originalnaivebayes5k.pickle","wb")
pickle.dump(classifier, save_classifier)
save_classifier.close()

Original Naive Bayes Algo accuracy percent: 72.7409638554217
Most Informative Features
                  boring = True              neg : pos    =     20.7 : 1.0
              engrossing = True              pos : neg    =     18.9 : 1.0
                 generic = True              neg : pos    =     16.4 : 1.0
                mediocre = True              neg : pos    =     16.4 : 1.0
                    loud = True              neg : pos    =     15.8 : 1.0
                    flat = True              neg : pos    =     15.1 : 1.0
              delightful = True              pos : neg    =     14.9 : 1.0
                 routine = True              neg : pos    =     14.4 : 1.0
               inventive = True              pos : neg    =     14.2 : 1.0
                  unique = True              pos : neg    =     13.6 : 1.0
              refreshing = True              pos : neg    =     12.3 : 1.0
               wonderful = True              pos : neg    =     12.1 : 1.0
             

In [7]:
MNB_classifier = SklearnClassifier(MultinomialNB())
MNB_classifier.train(training_set)
print("MNB_classifier accuracy percent:", (nltk.classify.accuracy(MNB_classifier, testing_set)) * 100)

save_classifier = open("pickled_algos/MNB_classifier5k.pickle", "wb")
pickle.dump(MNB_classifier, save_classifier)
save_classifier.close()

MNB_classifier accuracy percent: 71.23493975903614


In [8]:
voted_classifier = VoteClassifier(
    classifier,
    LinearSVC_classifier,
    MNB_classifier,
    BernoulliNB_classifier,
    LogisticRegression_classifier)

print("voted_classifier accuracy percent:", (nltk.classify.accuracy(voted_classifier, testing_set)) * 100)


def sentiment(text):
    feats = find_features(text)

    return voted_classifier.classify(feats)


voted_classifier accuracy percent: 73.19277108433735


In [9]:
import tweepy
from tweepy import Stream
from tweepy import OAuthHandler
from tweepy.streaming import StreamListener
import json
import sentiment_mod as s

#consumer key, consumer secret, access token, access secret.
ckey="MkFddAOkMPMZYAcNE1IcHusmz"
csecret="fJlyCNbZt7hcrMTlynbZWmP4R7h9oenhCvyJF8KEea0w20n8KD"
atoken="1411954077039222790-6nQZdgAYqiw4YBnpmYCQDZjxOzj1Cx"
asecret="kMlJx2MX62ucG7izINcyMxmO8s0ZoYTbqsbVcaq71k9Iu"

class listener(StreamListener):

    def on_data(self, data):
        try:
            all_data = json.loads(data)

            tweet = all_data["text"]
            sentiment_value, confidence = s.sentiment(tweet)
         
            print('\n')
            print(tweet)
            print('\n')
            print("******************************")
            print('\n')
            print(sentiment_value, confidence)
            print('\n')
            print("=======================================================================")
            print('\n')
            
            if confidence*100 >= 80:
                output = open("twitter-out.txt","a")
                output.write(sentiment_value)
                output.write('\n')
                output.close()

            return True
        except:
            return True

    def on_error(self, status):
        print(status)

auth = OAuthHandler(ckey, csecret)
auth.set_access_token(atoken, asecret)

twitterStream = Stream(auth, listener())
twitterStream.filter(track=["happy"])

10664


RT @kryptonitepost: I just wanna be happy. Not confused, not hurt, not stressed, just happy


******************************


neg 0.8






RT @Blessin80391771: Happy Birthday @Iam_IkeOnyema  wishing you more of God's grace and protection . Have fun https://t.co/bY7NJ94CfD


******************************


pos 1.0






RT @sbattrawden: Happy #freedomday everyone! Us NHS staff working in ICU tonight were very relieved to find out the COVID pandemic finished…


******************************


pos 0.8






RT @kaidennn__: happy birthday karl we love u 
rt to be swag 
#honktwtselfieday https://t.co/k6vtWSNaIn


******************************


pos 1.0






@Jayant_R_Patil @PrateekJPatil Happy birthday


******************************


pos 1.0






RT @WIZARD_BSC: LFG, wizards! We're happy to announce we had the best IDO launch IDO.
Endorsed by @BARMY_Official we are giving away $500 B…


******************************


pos 1.0






RT @mseiraa: Grabe, the power of Kisses 



RT @morales_ej: @JMDReid @creswelllyn @Simon_Lindley @snstreet @BellaSwannBooks @breezeemc @mfoulk50 @Tammysdragonfly @mcgrathbooks @EB_Whe…


******************************


neg 0.8






@sapsdnf @honkkarl HAPPY BIRTHDAY KARLOS


******************************


neg 0.8






@kkuunn2000 happy birthday ka! https://t.co/McasibPelq


******************************


pos 1.0






@TA3NGERINE Ahh happy ako para sayo!!!HAHAHA


******************************


pos 1.0






RT @Metalpanthers: It is likely completely irrelevant with Seattle rumored to be signing Driedger, but I’m happy to see Marchment get the r…


******************************


neg 1.0






RT @OneRepublic: We're happy we can finally announce this! The official release date for our album Human is August 27th. THANK YOU for your…


******************************


pos 1.0






RT @Kailee_love_: I woke up to 1000 followers on Twitter!!!!! Omg!!!! I’m so happy this Morning! 
Here’s a nip slip to celebrate!!!! 
#tues…



RT @skzkitty: anyway look how happy they are when knowing their dish is delicious and get praised for it &lt;3 https://t.co/OqfwRo4Hk3


******************************


pos 1.0






RT @sunpictures: Wishing our #Thalapathy65 director @NelsonDilpkumar a very Happy Birthday!

#HappyBirthdayNelson #HBDNelson https://t.co/s…


******************************


neg 1.0






Happy 3rd Anniversary Polca 💙
#เตนิว 
#โพก้า 
#HappyPolca3rdYear 
#ปีนี้โพก้า3ขวบ


******************************


neg 0.8






@HenshawKate @JulyetPeters @Dv_hairsalon @sabiegal Happy birthday to you great woman.u will continued to be a blessing to us


******************************


pos 1.0






RT @minilnat: เนื้อเพลงบางส่วนของ ‘Si Fueras Mia’ (If You Were Mine) 🌹

‘ถ้าเพียงแต่คุณเป็นของผม, ผมจะพาคุณไปในทุกๆที่ ที่จะทำให้คุณมีความ…


******************************


neg 0.8






RT @SangaMamiya: As you go about your business this week, may doors of opportunities open for you. May you find exceeding favour b



@indomyfess Happy birthday sayang ku 😍


******************************


pos 1.0






RT @ArmandoBrojaa: Happy to sign my new contract for @ChelseaFC, looking forward to the future💙 https://t.co/WvcSSQHhtH


******************************


neg 0.8






RT @sherinasinna: Kenal Zenius &amp; @sabdaps lebih dari 10 tahun yang lalu! 🤓 

Sejak itu kita sering ngobrolin tentang pendidikan &amp; bagaimana…


******************************


neg 0.8






RT @gt_bsecurity: Today we celebrate our CEO as he climbs a year older 🎂🎉 we pray that every dream, every goal will be achieved this new ye…


******************************


neg 1.0






Happy #MaxMonday everyone X https://t.co/PFFeBoVStc


******************************


neg 0.8






RT @yana0102618: Happy 200k to our newest furdad!

@jah447798 
PAGSI3OL EP
@SB19Official
#SB19GoUp2ndAnniv https://t.co/61RzxewFX9


******************************


neg 0.8






Entrepreneurs that they are I love my Dad happy Fathers day shirt . Ward

pos 1.0






@TKagubatan @ellacayabyab_ 39 Kween happy ako nakita kita at inum ka tubig lagi..pahinga ka muna

#EllaCayabyab | @ellacayabyab_


******************************


pos 1.0






@_who_me__ Wow beautiful view. Good morning Claire. Happy new week.


******************************


pos 1.0






@tabithalmao Happy early birthday Corey 🥳


******************************


pos 0.6






RT @Emy_san_arts: In your arms 💕

Happy Anniversary Final Fantasy X ✨
#FFX #FinalFantasyX #FF10_20th #FF10_20周年 https://t.co/mwJ0Zcb8Gu


******************************


neg 0.8






RT @goknursatur: Happy monday😎🏝🎶⛱🌺☕ https://t.co/mUQW82dLsL


******************************


neg 0.8






RT @WIZARD_BSC: LFG, wizards! We're happy to announce we had the best IDO launch IDO.
Endorsed by @BARMY_Official we are giving away $500 B…


******************************


pos 1.0






RT @0HSES00: THE SPANISH LYRICS “if you only were mine, the places I would take you to see you happy and see you smi

...

neg 0.8






RT @Wewillrocku66: @viericrespo @TheLadyDyan @SherylPrattERA @Cattalhina @Jimmysimmonds1 @_____YO_____YO @sherrig108 @BryanMatthews23 @Sath…


******************************


neg 0.8






Happy birthday @immewnity !! One of the coolest people in the Pokémon community and a consistent HUGE help to the c… https://t.co/skGYO442hC


******************************


neg 0.6






RT @AlankarMBFans: Advance happy birthday to our  little princess #Sitaraghattamaneni 🥰😻

#SarkaruVaariPaata https://t.co/zAQmHLdf21


******************************


pos 1.0






RT @kid67_: best feeling ever is having someone who appreciates you in the smallest things. accepts you in times of hardships. comforts you…


******************************


pos 1.0






RT @goanewshub: ..@AamAadmiParty leader @raghav_chadha accepts challenge by @CabralNilesh to debate over power. "I am happy to note that yo…


******************************


pos 1.0






RT @iAfe_: Do you ever???? get scared to get



RT @leejihoonfairy: 190923:To me who isn’t perfect, bc I have U who are so perfect I think I lived a grateful life

210526:Bc I'm always wi…


******************************


pos 1.0






RT @PhDJagriti: It just like :
Pitta: ओ हो 🤦🙆 ये 'मोना' 😍😍 भी न मेरे साथ अपनी फोटो भी शेयर कर दी...
अब लोग Confuse 😂😂 रहेंगे की कौन ज्यादा…


******************************


neg 1.0






RT @sbattrawden: Happy #freedomday everyone! Us NHS staff working in ICU tonight were very relieved to find out the COVID pandemic finished…


******************************


pos 0.8






RT @amid_temitayo: Happy birthday to my favorite female OAP, multi talented personality and splash fm presenter @iam_bibilicious  always wa…


******************************


pos 1.0






happy anniversary mwah semoga sehat selalu


******************************


pos 1.0






RT @AmBeR1994m: Those pictures from the nightclub almost brought me to tears &amp; I've not been clubbing for years. I'm so happy for this day.…






RT @BrianBanana1: Quote \ reply with your favorite pic of Erica Nlewedim !! 

Happy anniversary Elites!!
#1YearWithErica
#BeingAnElite
#Dea…


******************************


pos 0.6






RT @aliahdomino: TW// ED

The other day, I told my therapist “I’m starting to fit back into my pre-pregnancy clothes, and my brain is still…


******************************


neg 1.0






@kernowco good morning Colin. Happy Monday x


******************************


neg 0.6






RT @rocknrollgarag1: Happy 74 birthday to the amazing Queen guitarist Brian May! https://t.co/JfMNiWrkkM


******************************


pos 1.0






@MissPeter1 Happy birthday sbali wam enjoy your day God bless you 🎂🎉🎊🎁🥂🎈🌹❤️👑 https://t.co/IxlOMMsKXo


******************************


pos 1.0






RT @mingyuHQpics: so happy to hear that all svt members renew their contract 😭 carats need ot13 till the end, yes to going seventy 🙌😭 https…


******************************


pos 0.8






@bhogleharsha Sir, Happy Birth



RT @Ethel_official: Happy anniversary Nengi, wow can't believe it's one year already, it's just like yesterday, no regrets, stanning u give…


******************************


neg 0.6






RT @3plebim: @Oduware7 Happy Birthday @Oduware7 God bless you Abundantly Amen 💥🙏


******************************


neg 0.8






RT @TeamArmaanians: Happy birthday week @ArmaanMalik22! We all are SO excited for your special day, so here’s one cute surprise from all of…


******************************


pos 1.0






RT @Chinni58430322: Emotional at the same time excited happy everything #AshwinKumar  #EnnaSollaPogirai


******************************


pos 0.8






RT @MusicThaman: It’s from the Year 2000 till 2021 I have been witnessing @shankarshanmugh Sir 
the Way he thinks Imagine the Science &amp; lif…


******************************


pos 1.0






RT @min9yu_tttop: from gyuhao’s "please take care of us” to the general leader’s “I’ll protect you” 😭😭😭

🐸: please take care of us carats
🐶…


ProtocolError: ('Connection broken: IncompleteRead(0 bytes read)', IncompleteRead(0 bytes read))

In [10]:
userID = "Jvrc95"
auth = tweepy.OAuthHandler(ckey, csecret)
auth.set_access_token(atoken, asecret)
api = tweepy.API(auth)

tweets = api.user_timeline(screen_name=userID, 
                           count=200,
                           include_rts = False,
                           tweet_mode = 'extended'
                           )

In [11]:
print("USER ID: ", userID)
for info in tweets:
     print("ID: {}".format(info.id))
     print(info.created_at)
     print(info.full_text)
     print("\n")

USER ID:  Jvrc95
ID: 1416102199592210433
2021-07-16 18:27:28
Se te está saliendo la hipocresía, bro.


ID: 1414688421696573442
2021-07-12 20:49:38
La gente acomplejada nunca me dejará de dar risa.


ID: 1413599000331051010
2021-07-09 20:40:39
Me vale verga, puñetas.


ID: 1411654003201437704
2021-07-04 11:51:56
Mi compa el pendejo y sin webos.


ID: 1410891786176790529
2021-07-02 09:23:09
Nunca más.


ID: 1410868394920857604
2021-07-02 07:50:12
@ErnestHenS Sentiste el 3 stock?


ID: 1410341301699059714
2021-06-30 20:55:43
Vincit qui se vincit


ID: 1407472099367788546
2021-06-22 22:54:32
Doctora, bonita y más alta que yo. Alv


ID: 1406490298084069377
2021-06-20 05:53:13
I move slowly, only because I don’t have to move for anybody.


ID: 1404866579565748224
2021-06-15 18:21:08
Los veo en 3 semanas hdspm.


ID: 1403407764286812161
2021-06-11 17:44:19
@JuanOctavioAlv1 ¿De qué me perdí?


ID: 1402940247160328194
2021-06-10 10:46:34
Last time you gonna see a bad guy like this.


ID: 140208

In [12]:
import speechbrain as sb
from speechbrain.pretrained import EncoderDecoderASR

In [13]:
from speechbrain.dataio.dataio import read_audio
from IPython.display import Audio

In [14]:

asr_model = EncoderDecoderASR.from_hparams(source="speechbrain/asr-crdnn-rnnlm-librispeech", savedir="pretrained_models/asr-crdnn-rnnlm-librispeech")


C:\Users\donal\anaconda3\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


In [15]:
asr_model.transcribe_file(r"C:\Users\donal\Downloads\uwu\real-time-tweet-sentiment-analysis-master\grab.wav")

TypeError: Invalid file: WindowsPath('C:/Users/donal/Downloads/uwu/real-time-tweet-sentiment-analysis-master/grab.wav')

In [16]:
!pip install praw

In [26]:
import praw

reddit = praw.Reddit(client_id='6cOKIV7_OKBd_-6kqOxYOg',
                     client_secret='3xEPxxGZPrtLBaRXPmIG1HBEZOARwQ', password='Ece321ad!',
                     user_agent='uwu', username='onesleepybird')

In [27]:
subreddit = reddit.subreddit('python')

In [28]:
hot_python = subreddit.hot()

In [29]:
hot_python = subreddit.hot(limit=3)
for submission in hot_python:
    if not submission.stickied:
        print('Title: {}, ups: {}, downs: {}, Have we visited?: {}'.format(submission.title,
                                                                           submission.ups,
                                                                           submission.downs,
                                                                           submission.visited))

Title: My first big project in python (Im really proud of it!!!), ups: 461, downs: 0, Have we visited?: False


In [58]:
### Implementación reddit
### Buscar un 
import re
from pprint import pprint
import praw

r = praw.Reddit(client_id='6cOKIV7_OKBd_-6kqOxYOg',
                     client_secret='3xEPxxGZPrtLBaRXPmIG1HBEZOARwQ', password='Ece321ad!',
                     user_agent='uwu', username='onesleepybird')
submissions =  r.redditor("hoosakiwi").submissions.hot()

self_texts = []
for link in submissions:
    ## aquí ya se limpia el post/reddit
    print("----", ' '.join(re.sub("(/[A-Za-z0-9]+)|(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|(/+:\/\/\S+)"," ",link.title).split()), "----")
    print(' '.join(re.sub("(/[A-Za-z0-9]+)|(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",link.selftext).split()), "\n")


---- Trump Organization Is Charged in 15 Year Tax Scheme ----
 

---- New York Suspends Giuliani s Law License ----
 

---- Trump Justice Department monitored Washington Post reporters phone calls in 2017 ----
 

---- r Lounge ----
removed 

---- Requesting r Think we can use it to host predictions for the league of legends subreddit ----
 

---- Australia floods Thousands to be evacuated as downpours worsen ----
 

---- r is looking for new moderators ----
Edit We are no longer accepting applications If you do not hear from us in 2 3 weeks then we have decided not to move forward with your application nbsp Hello Everyone We are looking to recruit new mods to our team You do not need previous moderation experience but you should be willing to spend at minimum 7 hours a week helping to moderate the subreddit So what does that entail As a mod you will Review community reported posts and approve or remove them as per our rules Respond to modmails from the community Warn and ban users who 

In [78]:
import argparse
import json
import os
import random
import sys
import textwrap
from itertools import chain
from urllib.request import urlopen
from urllib.parse import urlencode


def get_api_key():
    try:
        return os.environ['YOUTUBE_API_KEY']
    except KeyError:
        msg = '''You must create and export a YOUTUBE_API_KEY, instructions:
    1. Go-to https://console.developers.google.com/apis/credentials
    2. Create credentials with an "API Key" type
    3. export YOUTUBE_API_KEY=<your key goes here>'''
        print(msg)
        sys.exit(1)


def check_youtube_video_id(str):
    str = str.strip()

    # This is a bit naive but it should be good enough to protect against
    # accidentally pasting in the wrong value.
    if len(str) == 11:
        return str
    else:
        msg = f'not a valid YouTube video id: "{str}"'
        raise argparse.ArgumentTypeError(msg)


def check_positive_int(val):
    int_val = int(val)

    if int_val >= 0:
        return int_val
    else:
        msg = f'must be a positive integer: "{val}"'
        raise argparse.ArgumentTypeError(msg)


def check_omit_authors(omit_authors):
    if omit_authors == '':
        return []

    return list(map(str.strip, omit_authors.split(',')))


def display_names(results, is_verbose):
    authors = []

    for item in results['items']:
        comment = item['snippet']['topLevelComment']['snippet']
        author = comment['authorDisplayName']
        authors.append(author)

        if is_verbose:
            print(f'  {author}')

    return authors


def progress(page_count):
    if page_count > 1:
        print('')

    print(f'Getting comments for page {page_count}...')

    return page_count + 1


def get_comments(api_params):
    api_endpoint = 'https://www.googleapis.com/youtube/v3/commentThreads'
    encoded_params = urlencode(api_params)

    with urlopen(f'{api_endpoint}?{encoded_params}') as response:
        return json.load(response)


def get_comment_authors(api_token, video_id, is_verbose):
    authors = []
    page_count = 1

    api_params = {
        'key': api_token,
        'part': 'snippet',
        'videoId': video_id,
        'maxResults': 100,
    }

    results = get_comments(api_params)
    page_count = progress(page_count)
    authors.append(display_names(results, is_verbose))

    next_page_token = results.get('nextPageToken')

    while next_page_token:
        page_count = progress(page_count)

        api_params['pageToken'] = next_page_token
        results = get_comments(api_params)
        authors.append(display_names(results, is_verbose))

        next_page_token = results.get('nextPageToken')

    return authors


def flatten_list(items):
    return list(chain.from_iterable(items))


def sorted_unique_list(items):
    return sorted(list(set(items)))


def remove_authors(items, skip_items):
    #  This could be rewritten as a list comprehension such as:
    #   return [item.strip() for item in items if item not in skip_items]
    #
    # But IMO this approach is much more readable.
    for item in skip_items:
        if item.strip() in items:
            items.remove(item)

    return items


def pick_winners(authors, authors_count, winner_count):
    # We can't pick more winners than we have in total.
    if winner_count > authors_count:
        winner_count = authors_count

    return (random.sample(authors, winner_count), winner_count)


def parseargs():
    parser = argparse.ArgumentParser(
        formatter_class=argparse.RawDescriptionHelpFormatter,
        description=textwrap.dedent('''\
        Get a list of top level comments from a YouTube video and then
        pick N amount of unique comment authors by choosing them randomly.
        '''))

    parser.add_argument('video_id', default=None,
                        metavar='VIDEO_ID', type=check_youtube_video_id,
                        help='the 11 characters after ?v= in a YouTube URL')

    parser.add_argument('--winners', default=10, type=check_positive_int,
                        metavar='WINNERS',
                        help='number of winners to pick (defaults to 10)')

    parser.add_argument('--omit-authors', default='', type=check_omit_authors,
                        metavar='OMIT_AUTHORS',
                        help='comma separated list of author names to omit')

    parser.add_argument('--verbose', default=False, type=bool, nargs='?',
                        const=True, metavar='BOOL',
                        help='output author display names during the progress')

    return parser.parse_args()


def generate_report(winners, winner_count, authors_count,
                    duplicate_authors_count, omit_authors_count,
                    authors_final_count):
    winners = '\n  '.join(winners)

    print(f'''
{authors_count} top level comments were returned
- {duplicate_authors_count} comment(s) had duplicate authors
- {omit_authors_count} comment authors were explicitly omit
= {authors_final_count} comment authors have a chance to win
Winners ({winner_count}):
  {winners}''')

    return None


if __name__ == '__main__':
    args = parseargs()

    omit_authors_count = len(args.omit_authors)

    authors = get_comment_authors(get_api_key(), args.video_id, args.verbose)
    authors = flatten_list(authors)
    authors_count = len(authors)

    authors = sorted_unique_list(authors)
    authors_unique_count = len(authors)

    authors = remove_authors(authors, args.omit_authors)
    authors_final_count = len(authors)

    duplicate_authors_count = authors_count - authors_unique_count

    winners, winner_count = pick_winners(authors, authors_final_count,
                                         args.winners)

    generate_report(winners=winners,
                    winner_count=winner_count,
                    authors_count=authors_count,
                    duplicate_authors_count=duplicate_authors_count,
                    omit_authors_count=omit_authors_count,
                    authors_final_count=authors_final_count)

usage: ipykernel_launcher.py [-h] [--winners WINNERS]
                             [--omit-authors OMIT_AUTHORS] [--verbose [BOOL]]
                             VIDEO_ID
ipykernel_launcher.py: error: argument VIDEO_ID: not a valid YouTube video id: "C:\Users\donal\AppData\Roaming\jupyter\runtime\kernel-a7b534df-1a70-42e4-9643-8902d16b4e90.json"
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\donal\anaconda3\lib\argparse.py", line 2422, in _get_value
    result = type_func(arg_string)
  File "<ipython-input-78-f4cf9078cd6c>", line 34, in check_youtube_video_id
    raise argparse.ArgumentTypeError(msg)
argparse.ArgumentTypeError: not a valid YouTube video id: "C:\Users\donal\AppData\Roaming\jupyter\runtime\kernel-a7b534df-1a70-42e4-9643-8902d16b4e90.json"

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\donal\anaconda3\lib\argparse.py", line 1800, in parse_known_args
    namespace, args = self._parse_known_args(args, namespace)
  File "C:\Users\donal\anaconda3\lib\argparse.py", line 2009, in _parse_known_args
    stop_index = consume_positionals(start_index)
  File "C:\Users\donal\anaconda3\lib\argparse.py", line 1965, in consume_positionals
    take_action(action, args)
  File "C:\Users\donal\anaconda3\lib\argparse.py", line 1858, in take_action
    a

TypeError: object of type 'NoneType' has no len()